In [ ]:
import os

root_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))

from tqdm.notebook import tqdm
from transformers import BatchEncoding,WordpieceTokenizer
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import pickle
import collections


In [ ]:
livedoor_path = os.path.join(root_dir,'data/livedoor')
train_path = os.path.join(livedoor_path,'namco_train_tokenized.tsv')
val_path = os.path.join(livedoor_path,'namco_dev_tokenized.tsv')
test_path = os.path.join(livedoor_path,'namco_test_tokenized.tsv')


In [ ]:
def read_livedoor(path):
    all_labels = ['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']
    data_ = open(path,encoding='utf8').readlines()
    texts = []
    labels = []
    for l in data_:
        l = l.strip().split('\t')
        texts.append(l[0])
        labels.append(int(l[1]))
    return texts,labels


In [ ]:
train_texts, train_labels = read_livedoor(train_path)
val_texts, val_labels = read_livedoor(val_path)
test_texts, test_labels = read_livedoor(test_path)


In [ ]:
def convert_token_to_id(vocab_index,token,unk_token='[UNK]'):
        """Converts a token (str/unicode) to an id using the vocab."""
        if token[:2] == '##':
            token = token[2:]

        return vocab_index.get(token, vocab_index.get(unk_token))

def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  with open(vocab_file, encoding='utf8') as reader:
    while True:
      token = reader.readline()
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab

def texts2encoding(texts,vocab_file,do_lower_case=True,max_seq_len=512):
    input_ids = []
    attention_mask = []
    for text in tqdm(texts):
        features = text2feature(text,vocab_file,do_lower_case=True,max_seq_len=512)
        input_ids.append(features[0])
        attention_mask.append(features[1])
          
    tmp_dic = {'input_ids':input_ids,'attention_mask':attention_mask}
    encoding = BatchEncoding(tmp_dic)
    
    return encoding

def text2feature(text,vocab_file,do_lower_case=True,max_seq_len=512):
    vocab_index = load_vocab(vocab_file)
    tokens = text.strip().split()
    
    tokens = ['[CLS]']+tokens
    tokens.append('[SEP]')
    _input_ids = [convert_token_to_id(vocab_index, token) for token in tokens]
    _attention_mask = [1]*len(_input_ids)

    assert len(_input_ids)<=max_seq_len
    while len(_input_ids)<max_seq_len:
        _input_ids.append(0)
        _attention_mask.append(0)

    return _input_ids,_attention_mask



In [ ]:
vocab_file = os.path.join(root_dir,'model/namco_distilbert/vocab-lower.txt')

train_encodings = texts2encoding(train_texts,vocab_file)
val_encodings = texts2encoding(val_texts,vocab_file)
test_encodings = texts2encoding(test_texts,vocab_file)

train_encodings_path = os.path.join(livedoor_path,'namco_train_encodings.pickle')
val_encodings_path = os.path.join(livedoor_path,'namco_dev_encodings.pickle')
test_encodings_path = os.path.join(livedoor_path,'namco_test_encodings.pickle')

pickle.dump(train_encodings,open(train_encodings_path,'wb'))
pickle.dump(val_encodings,open(val_encodings_path,'wb'))
pickle.dump(test_encodings,open(test_encodings_path,'wb'))
                          

In [ ]:
class LivedoorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = pickle.load(open(train_encodings_path,'rb'))
val_encodings = pickle.load(open(val_encodings_path,'rb'))
test_encodings = pickle.load(open(test_encodings_path,'rb'))
    
train_dataset = LivedoorDataset(train_encodings, train_labels)
val_dataset = LivedoorDataset(val_encodings, val_labels)
test_dataset = LivedoorDataset(test_encodings, test_labels)

train_dataset_path = os.path.join(livedoor_path,'namco_train_dataset.pickle')
val_dataset_path = os.path.join(livedoor_path,'namco_dev_dataset.pickle')
test_dataset_path = os.path.join(livedoor_path,'namco_test_dataset.pickle')

pickle.dump(train_dataset,open(train_dataset_path,'wb'))
pickle.dump(val_dataset,open(val_dataset_path,'wb'))
pickle.dump(test_dataset,open(test_dataset_path,'wb'))


In [ ]:
train_dataset = pickle.load(open(train_dataset_path,'rb'))
val_dataset = pickle.load(open(val_dataset_path,'rb'))
test_dataset = pickle.load(open(test_dataset_path,'rb'))


In [ ]:
# the num_labels in config.json should be 9 for livedoor news classification task
# rename config_livedoor.json as config.json for this task
# and use the following code to make sure model.num_labels is equal to 9

'''
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model_path = os.path.join(root_dir,'model/namco_distilbert')
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.num_labels
'''


In [ ]:
training_args = TrainingArguments(
    output_dir=os.path.join(root_dir,'model/namco_distilbert/output_livedoor'),          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join(root_dir,'model/namco_distilbert/output_livedoor'),            # directory for storing logs
    logging_steps=100,
)

model_path = os.path.join(root_dir,'model/namco_distilbert')
model = DistilBertForSequenceClassification.from_pretrained(model_path)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()


In [ ]:
training_args = TrainingArguments(
    output_dir=os.path.join(root_dir,'model/namco_distilbert/output_livedoor'),          # output directory
    num_train_epochs=0,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join(root_dir,'model/namco_distilbert/output_livedoor'),            # directory for storing logs
    logging_steps=10,
)

model_path = os.path.join(root_dir,'model/namco_distilbert/output_livedoor/checkpoint-5500')
model = DistilBertForSequenceClassification.from_pretrained(model_path)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
)

ori_predictions = trainer.predict(test_dataset=test_dataset).predictions


In [ ]:
import numpy as np

predictions = np.argmax(ori_predictions, axis=1)
#print(predictions)
correct_labels = test_dataset.labels

print(len(predictions))
print(len(correct_labels))
cor = 0
for i in range(len(predictions)):
  if predictions[i]==correct_labels[i]:
    cor += 1
  #print(predictions[i],correct_labels[i])
print(cor/len(predictions))